# Python libraries

In [1]:
import ee
import geemap
import os
import numpy as np
import matplotlib.pyplot as plt

# Display map
To display the map the Map of geemap is used https://github.com/giswqs/geemap, which inherits from ipyleaft.Map https://ipyleaflet.readthedocs.io/en/latest/api_reference/map.html.

In [2]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

# Load Landsat 8 image
To load a the image from Landsat 8, a image from Google Earth Engine Data Catalog is filtered https://developers.google.com/earth-engine/datasets/catalog/landsat-8/ and displayed on the interactive map.

In [25]:
lon=-75.32;
lat=4.88;

point = ee.Geometry.Point(lon, lat);
start = ee.Date('2013-02-11');
finish = ee.Date('2021-05-30');

# Filter a image from the landsat raw images collection, using bounds, date, and selecting the image with the least cloud cover
img = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
            .filterBounds(point) \
            .filterDate(start, finish) \
            .sort('CLOUD_COVER', True) \
            .first()

# Select the corresponding bands for the multiespectral and panchromatic image
img_multi = img.select(['B4', 'B3', 'B2'])
img_pan = img.select(['B8'])

# Display the images on the map
Map.addLayer(img_multi)
Map.addLayer(img_pan)

# Center the map
Map.setCenter(lon, lat, 12)

# B. Select a roi using the coordinates
Instead of drawing the rectangle, you can select an area by drawing a polygon using the GeoJson convention https://tools.ietf.org/html/rfc7946#section-3.1.6. There is a polygon drawn by default.

In [24]:
delta=0.1;

P1=[lon-delta, lat-delta];
P2=[lon+delta, lat+delta];


# feature to extract
roi = ee.Geometry.Rectangle([P1, P2]);

# add the polygon to the map
Map.addLayer(roi, {}, 'feature');
print('Polygon coordinates >>', roi.getInfo().get('coordinates'))

Polygon coordinates >> [[[-75.41999999999999, 4.78], [-75.22, 4.78], [-75.22, 4.9799999999999995], [-75.41999999999999, 4.9799999999999995], [-75.41999999999999, 4.78]]]


# Download the roi as a TIFF file
The next block generates the path to store the TIFF.

In [26]:
out_dir = os.path.join(os.path.expanduser('~'), 'Proyectos/Universidad/Maestria/trozos/SierraNevada')
filename_xs = os.path.join(out_dir, 'landsat_multi.tif')
filename_pan = os.path.join(out_dir, 'landsat_pan.tif')

### Download as raw images without resizing

* A TIFF file including all bands

In [27]:
geemap.ee_export_image(img_multi, filename=filename_xs, region=roi, file_per_band=False)
geemap.ee_export_image(img_pan, filename=filename_pan, region=roi, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\edwin.hurtado\Proyectos\Universidad\Maestria\trozos\SierraNevada\landsat_multi.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\edwin.hurtado\Proyectos\Universidad\Maestria\trozos\SierraNevada\landsat_pan.tif


* A TIFF file for each band

In [28]:
geemap.ee_export_image(img_multi, filename=filename_xs, region=roi, file_per_band=True)
geemap.ee_export_image(img_pan, filename=filename_pan, region=roi, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\edwin.hurtado\Proyectos\Universidad\Maestria\trozos\SierraNevada
Generating URL ...
Please wait ...
Data downloaded to C:\Users\edwin.hurtado\Proyectos\Universidad\Maestria\trozos\SierraNevada\landsat_pan.tif


### Download resized images

* A TIFF file including all bands

In [33]:
# Set the image scale meters, 150 means 15 meters of resolution, change to the convenient value
img_scale = 10
geemap.ee_export_image(img_multi, filename=filename_xs, scale = img_scale, region=roi, file_per_band=False)
geemap.ee_export_image(img_pan, filename=filename_pan, scale = img_scale, region=roi, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\edwin.hurtado\Proyectos\Universidad\Maestria\trozos\SierraNevada\landsat_multi.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\edwin.hurtado\Proyectos\Universidad\Maestria\trozos\SierraNevada\landsat_pan.tif


* A TIFF file for each band (This option is recommended to proceed with the pansharpening module)

In [34]:
# Set the image scale meters, 150 means 15 meters of resolution, change to the convenient value
img_scale = 10
geemap.ee_export_image(img_multi, filename=filename_xs, scale = img_scale, region=roi, file_per_band=True)
geemap.ee_export_image(img_pan, filename=filename_pan, scale = img_scale, region=roi, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\edwin.hurtado\Proyectos\Universidad\Maestria\trozos\SierraNevada
Generating URL ...
Please wait ...
Data downloaded to C:\Users\edwin.hurtado\Proyectos\Universidad\Maestria\trozos\SierraNevada\landsat_pan.tif


2013-01-01_2014-01-01
2014-01-01_2015-01-01
2015-01-01_2016-01-01
2016-01-01_2017-01-01
2017-01-01_2018-01-01
2018-01-01_2019-01-01
2019-01-01_2020-01-01
2020-01-01_2021-01-01


In [48]:
# ALGORITMO DE DESCARGA

lon=-75.32;
lat=4.88;

dates = ['2013-01-01','2014-01-01','2015-01-01','2016-01-01','2017-01-01','2018-01-01','2019-01-01','2020-01-01','2021-01-01']

point = ee.Geometry.Point(lon, lat);

for i in range(len(dates)-1):

    start = ee.Date(dates[i]);
    finish = ee.Date(dates[i+1]);

    # Filter a image from the landsat raw images collection, using bounds, date, and selecting the image with the least cloud cover
    img = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
                .filterBounds(point) \
                .filterDate(start, finish) \
                .sort('CLOUD_COVER', True) \
                .median()

    # Select the corresponding bands for the multiespectral and panchromatic image
    img_multi = img.select(['B4', 'B3', 'B2'])
    img_pan = img.select(['B8'])

    delta=0.1;

    P1=[lon-delta, lat-delta];
    P2=[lon+delta, lat+delta];


    # feature to extract
    roi = ee.Geometry.Rectangle([P1, P2]);

    # add the polygon to the map
    Map.addLayer(roi, {}, 'feature');
    print('Polygon coordinates >>', roi.getInfo().get('coordinates'))

    # Display the images on the map
    Map.addLayer(img_multi)
    Map.addLayer(img_pan)

    # Center the map
    Map.setCenter(lon, lat, 12)

    out_dir = os.path.join(os.path.expanduser('~'), 'Proyectos/Universidad/Maestria/trozos/SierraNevada')
    filename_xs = os.path.join(out_dir, 'landsat_multi'+dates[i]+"_"+dates[i+1]+'.tif')
    filename_pan = os.path.join(out_dir, 'landsat_pan'+dates[i]+"_"+dates[i+1]+'.tif')

    img_scale = 20
    geemap.ee_export_image(img_multi, filename=filename_xs, scale = img_scale, region=roi, file_per_band=False)
    geemap.ee_export_image(img_multi, filename=filename_xs, scale = img_scale, region=roi, file_per_band=True)
    geemap.ee_export_image(img_pan, filename=filename_pan, scale = img_scale, region=roi, file_per_band=False)

Polygon coordinates >> [[[-75.41999999999999, 4.78], [-75.22, 4.78], [-75.22, 4.9799999999999995], [-75.41999999999999, 4.9799999999999995], [-75.41999999999999, 4.78]]]
Generating URL ...
Please wait ...
Data downloaded to C:\Users\edwin.hurtado\Proyectos\Universidad\Maestria\trozos\SierraNevada\landsat_multi2013-01-01_2014-01-01.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\edwin.hurtado\Proyectos\Universidad\Maestria\trozos\SierraNevada
Generating URL ...
Please wait ...
Data downloaded to C:\Users\edwin.hurtado\Proyectos\Universidad\Maestria\trozos\SierraNevada\landsat_pan2013-01-01_2014-01-01.tif
Polygon coordinates >> [[[-75.41999999999999, 4.78], [-75.22, 4.78], [-75.22, 4.9799999999999995], [-75.41999999999999, 4.9799999999999995], [-75.41999999999999, 4.78]]]
Generating URL ...
Please wait ...
Data downloaded to C:\Users\edwin.hurtado\Proyectos\Universidad\Maestria\trozos\SierraNevada\landsat_multi2014-01-01_2015-01-01.tif
Generating URL ...
Please wait ..

Generating URL ...
Please wait ...
Data downloaded to C:\Users\edwin.hurtado\Proyectos\Universidad\Maestria\trozos\SierraNevada\landsat_multi2020-01-01_2021-01-01.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\edwin.hurtado\Proyectos\Universidad\Maestria\trozos\SierraNevada
Generating URL ...
Please wait ...
Data downloaded to C:\Users\edwin.hurtado\Proyectos\Universidad\Maestria\trozos\SierraNevada\landsat_pan2020-01-01_2021-01-01.tif
